## Quantum Kitchen Sinks by Wilson et al.[1]

### Algorithm:

Given a dataset $\{y_i, u_i\}_{i=1..m}$ of $m$ points where $u_i \in \mathbb{R}^p$ and a set of $E$ feature vectors derived from $u$, also called 'episodes'[1]. The E.R.M is:
$$arg\min\limits_{\sigma \in \mathbb{R}} \sum\limits_{e=1}^{|E|} \frac{1}{m}\sum\limits_{i=1}^{M}c(\Phi_e(\theta_e, u_i), y_i)$$

Where $c$ is the cost function, where $\theta_e$ are the gate parameters for each episode, 
$$\Omega_e \in \mathbb{R}^{q \times p}; \beta_e \in \mathbb{R}^q$$
$$\theta_{i, e} = \Omega_eu_i + \beta_e$$
with $$u \in \mathbb{R}^{n\times p}; \theta_e = u \Omega_e^T + \beta_e$$
$$\Omega_e \sim \mathcal{N}(0, \sigma^2)); \beta_e \sim \mathcal{U}(0, 2\pi)$$

In [3]:
from cirq.contrib.svg import SVGCircuit
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error
from mnist import MNISTData

import scipy
import cirq
import numpy as np
import tensorflow_quantum as tfq
import tensorflow as tf

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
mnist_data = MNISTData(seed)
X_train, X_test, y_train, y_test = mnist_data.get_three_five_test_train_split()
train_indices = np.random.randint(0, X_train.shape[0], 1600)
test_indices = np.random.randint(0, X_test.shape[0], 400)

X_tr = X_train[train_indices]
y_tr = y_train[train_indices]
X_te = X_test[test_indices]
y_te = y_test[test_indices]
print(X_tr.shape, X_te.shape)

(1600, 784) (400, 784)


In [52]:
class TwoQubitQks:
    def __init__(self, X_train, y_train, X_test, y_test, episodes=100):
        self.qubits = cirq.LineQubit.range(2)
        # the number of qubits
        self.q = 2
        self.p = X_train.shape[1]
        self.r = self.p/self.q
        self.E = episodes
        self.X_train = X_train
        self.y_train = y_train
        self.y_test = y_test
        self.X_test = X_test
        # mask = q * p
        mask = np.ones((self.q, self.p))
        mask[0,:int(self.r)], mask[1,int(self.r):] = 0.0, 0.0
        self.mask = mask.reshape(self.q*self.p)
    
    def _get_ansatz(self, theta, draw=False):
        circuit = cirq.Circuit()
        circuit.append(cirq.rx(theta[0])(self.qubits[0]))
        circuit.append(cirq.rx(theta[1])(self.qubits[1]))
        circuit.append(cirq.CNOT(self.qubits[0], self.qubits[1]))
        circuit.append(cirq.measure(self.qubits[1]))
        
        if draw:
            SVGCircuit(circuit)    
        return circuit
    
    def _get_meas(self, theta):
        circuit = self._get_ansatz(theta)
        result = cirq.Simulator().run(circuit)
        return -1 if result.measurements['1'][0][0] == 0 else 1
    
    def _get_omega_and_beta(self, variance):
        stddev = variance ** 2
        # mask = E x q x p
        mask = self.mask.repeat(self.E)
        # mask = (E x (q*p))
        mask = mask.reshape((self.E, self.q*self.p))
        # omega_e = (E x (q*p))
        omega_e = np.random.normal(0.0, stddev, (self.E, (self.q * self.p)))
        # omega = (mask * omega_e) so (E x (q*p))
        omega = mask * omega_e
        # omega = (E x p x q)
        omega = omega.reshape((self.E, self.p, self.q), order='F')
        # beta = (E x q)
        beta = np.random.uniform(0.0, 2 * np.pi, (self.E, self.q))
        return omega, beta
    
    def _get_embeddings(self, variance):
        omega, beta = self._get_omega_and_beta(variance)
        # params = (n x E x q)
        params = self.X_train.dot(omega) + beta
        # params = (n * E) x q
        return params.reshape((params.shape[0] * params.shape[1], params.shape[2]))
        
    def _loss(self, variance):
        # labels = (n * E)
        labels = np.tile(self.y_train, self.E)
        params = self._get_embeddings(variance)
        
        # get predictions
        preds = np.array([self._get_meas(param) for param in params])
        # get loss
        return mean_squared_error(preds, labels)
    
    def train(self):
        result = scipy.optimize.minimize(self._loss, x0=1.0, method="COBYLA")
        self.variance = result['x']
        print(result)
        
    def test_accuracy(self):
        stddev = self.variance ** 2
        n = self.X_test.shape[0]
        omega = np.random.normal(0.0, stddev, (self.p, self.q)) * self.mask.reshape((self.p, self.q))
        beta = np.random.uniform(0.0, 2 * np.pi, (self.q,))
        
        params = self.X_test.dot(omega)
        preds = np.array([self._get_meas(param) for param in params])
        return (np.sum(preds == self.y_test)/preds.shape[0]) * 100

In [53]:
twoQubitQks = TwoQubitQks(X_tr, y_tr, X_te, y_te, episodes=10)

In [49]:
twoQubitQks._get_ansatz([np.pi, np.pi], draw=True)

0: ───Rx(π)───@───────
              │
1: ───Rx(π)───X───M───

We train the circuit over 1600 samples and test over 400 samples from the test set. We only optimize over the variance  of the normal distribution in this example. Wilson et al. achieved over 96% accuracy when trained over 10K episodes, this example is for demonstrating QKS and we did not train the model fully in the intrests of time. 

In [50]:
twoQubitQks.train()

     fun: 2.00525
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 23
  status: 1
 success: True
       x: array(-1.08193125)


In [ ]:
twoQubitQks.test_accuracy()

### References:

[1] Wilson, C.M., Otterbach, J.S., Tezak, N., Smith, R.S., Polloreno, A.M., Karalekas, P.J., Heidel, S., Alam, M.S., Crooks, G.E. and da Silva, M.P., 2018. Quantum kitchen sinks: An algorithm for machine learning on near-term quantum computers. arXiv preprint arXiv:1806.08321.